In [2]:
!python3 -V

Python 3.10.12


# Setup

## Dependencies

In [3]:

#!pip install transformer_lens
#!pip install gradio

In [4]:
import transformer_lens
from transformer_lens import HookedTransformer, utils
import torch
import numpy as np
import gradio as gr
import pprint
import json
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from huggingface_hub import HfApi
from IPython.display import HTML
from functools import partial
import tqdm.notebook as tqdm
import plotly.express as px
import pandas as pd

## Defining the Autoencoder

In [5]:
cfg = {
    "seed": 49,
    "batch_size": 4096,
    "buffer_mult": 384,
    "lr": 1e-4,
    "num_tokens": int(2e9),
    "l1_coeff": 3e-4,
    "beta1": 0.9,
    "beta2": 0.99,
    "dict_mult": 8,
    "seq_len": 128,
    "d_mlp": 2048,
    "enc_dtype":"fp32",
    "remove_rare_dir": False,
}
cfg["model_batch_size"] = 64
cfg["buffer_size"] = cfg["batch_size"] * cfg["buffer_mult"]
cfg["buffer_batches"] = cfg["buffer_size"] // cfg["seq_len"]

In [6]:
DTYPES = {"fp32": torch.float32, "fp16": torch.float16, "bf16": torch.bfloat16}
class AutoEncoder(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        d_hidden = cfg["d_mlp"] * cfg["dict_mult"]
        d_mlp = cfg["d_mlp"]
        l1_coeff = cfg["l1_coeff"]
        dtype = DTYPES[cfg["enc_dtype"]]
        torch.manual_seed(cfg["seed"])
        self.W_enc = nn.Parameter(torch.nn.init.kaiming_uniform_(torch.empty(d_mlp, d_hidden, dtype=dtype)))
        self.W_dec = nn.Parameter(torch.nn.init.kaiming_uniform_(torch.empty(d_hidden, d_mlp, dtype=dtype)))
        self.b_enc = nn.Parameter(torch.zeros(d_hidden, dtype=dtype))
        self.b_dec = nn.Parameter(torch.zeros(d_mlp, dtype=dtype))

        self.W_dec.data[:] = self.W_dec / self.W_dec.norm(dim=-1, keepdim=True)

        self.d_hidden = d_hidden
        self.l1_coeff = l1_coeff

        self.to("cuda")

    def forward(self, x):
        x_cent = x - self.b_dec
        acts = F.relu(x_cent @ self.W_enc + self.b_enc)
        x_reconstruct = acts @ self.W_dec + self.b_dec
        l2_loss = (x_reconstruct.float() - x.float()).pow(2).sum(-1).mean(0)
        l1_loss = self.l1_coeff * (acts.float().abs().sum())
        loss = l2_loss + l1_loss
        return loss, x_reconstruct, acts, l2_loss, l1_loss

    @torch.no_grad()
    def remove_parallel_component_of_grads(self):
        W_dec_normed = self.W_dec / self.W_dec.norm(dim=-1, keepdim=True)
        W_dec_grad_proj = (self.W_dec.grad * W_dec_normed).sum(-1, keepdim=True) * W_dec_normed
        self.W_dec.grad -= W_dec_grad_proj

    # def get_version(self):
    #     return 1+max([int(file.name.split(".")[0]) for file in list(SAVE_DIR.iterdir()) if "pt" in str(file)])

    # def save(self):
    #     version = self.get_version()
    #     torch.save(self.state_dict(), SAVE_DIR/(str(version)+".pt"))
    #     with open(SAVE_DIR/(str(version)+"_cfg.json"), "w") as f:
    #         json.dump(cfg, f)
    #     print("Saved as version", version)

    # def load(cls, version):
    #     cfg = (json.load(open(SAVE_DIR/(str(version)+"_cfg.json"), "r")))
    #     pprint.pprint(cfg)
    #     self = cls(cfg=cfg)
    #     self.load_state_dict(torch.load(SAVE_DIR/(str(version)+".pt")))
    #     return self

    @classmethod
    def load_from_hf(cls, version):
        """
        Loads the saved autoencoder from HuggingFace.

        Version is expected to be an int, or "run1" or "run2"

        version 25 is the final checkpoint of the first autoencoder run,
        version 47 is the final checkpoint of the second autoencoder run.
        """
        if version=="run1":
            version = 25
        elif version=="run2":
            version = 47

        cfg = utils.download_file_from_hf("NeelNanda/sparse_autoencoder", f"{version}_cfg.json")
        pprint.pprint(cfg)
        self = cls(cfg=cfg)
        self.load_state_dict(utils.download_file_from_hf("NeelNanda/sparse_autoencoder", f"{version}.pt", force_is_torch=True))
        return self


## Utils

### Get Reconstruction Loss

In [7]:
def replacement_hook(mlp_post, hook, encoder):
    mlp_post_reconstr = encoder(mlp_post)[1]
    return mlp_post_reconstr

def mean_ablate_hook(mlp_post, hook):
    mlp_post[:] = mlp_post.mean([0, 1])
    return mlp_post

def zero_ablate_hook(mlp_post, hook):
    mlp_post[:] = 0.
    return mlp_post

@torch.no_grad()
def get_recons_loss(num_batches=5, local_encoder=None):
    if local_encoder is None:
        local_encoder = encoder
    loss_list = []
    for i in range(num_batches):
        tokens = all_tokens[torch.randperm(len(all_tokens))[:cfg["model_batch_size"]]]
        loss = model(tokens, return_type="loss")
        recons_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=[(utils.get_act_name("post", 0), partial(replacement_hook, encoder=local_encoder))])
        # mean_abl_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=[(utils.get_act_name("post", 0), mean_ablate_hook)])
        zero_abl_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=[(utils.get_act_name("post", 0), zero_ablate_hook)])
        loss_list.append((loss, recons_loss, zero_abl_loss))
    losses = torch.tensor(loss_list)
    loss, recons_loss, zero_abl_loss = losses.mean(0).tolist()

    print(f"loss: {loss:.4f}, recons_loss: {recons_loss:.4f}, zero_abl_loss: {zero_abl_loss:.4f}")
    score = ((zero_abl_loss - recons_loss)/(zero_abl_loss - loss))
    print(f"Reconstruction Score: {score:.2%}")
    # print(f"{((zero_abl_loss - mean_abl_loss)/(zero_abl_loss - loss)).item():.2%}")
    return score, loss, recons_loss, zero_abl_loss

### Get Frequencies

In [8]:
# Frequency
@torch.no_grad()
def get_freqs(num_batches=25, local_encoder=None):
    if local_encoder is None:
        local_encoder = encoder
    act_freq_scores = torch.zeros(local_encoder.d_hidden, dtype=torch.float32).cuda()
    total = 0
    for i in tqdm.trange(num_batches):
        tokens = all_tokens[torch.randperm(len(all_tokens))[:cfg["model_batch_size"]]]

        _, cache = model.run_with_cache(tokens, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
        mlp_acts = cache[utils.get_act_name("post", 0)]
        mlp_acts = mlp_acts.reshape(-1, d_mlp)

        hidden = local_encoder(mlp_acts)[2]

        act_freq_scores += (hidden > 0).sum(0)
        total+=hidden.shape[0]
    act_freq_scores /= total
    num_dead = (act_freq_scores==0).float().mean()
    print("Num dead", num_dead)
    return act_freq_scores

## Visualise Feature Utils

In [9]:
from html import escape
import colorsys

from IPython.display import display

SPACE = "·"
NEWLINE="↩"
TAB = "→"

def create_html(strings, values, max_value=None, saturation=0.5, allow_different_length=False, return_string=False):
    # escape strings to deal with tabs, newlines, etc.
    escaped_strings = [escape(s, quote=True) for s in strings]
    processed_strings = [
        s.replace("\n", f"{NEWLINE}<br/>").replace("\t", f"{TAB}&emsp;").replace(" ", "&nbsp;")
        for s in escaped_strings
    ]

    if isinstance(values, torch.Tensor) and len(values.shape)>1:
        values = values.flatten().tolist()

    if not allow_different_length:
        assert len(processed_strings) == len(values)

    # scale values
    if max_value is None:
        max_value = max(max(values), -min(values))+1e-3
    scaled_values = [v / max_value * saturation for v in values]

    # create html
    html = ""
    for i, s in enumerate(processed_strings):
        if i<len(scaled_values):
            v = scaled_values[i]
        else:
            v = 0
        if v < 0:
            hue = 0  # hue for red in HSV
        else:
            hue = 0.66  # hue for blue in HSV
        rgb_color = colorsys.hsv_to_rgb(
            hue, v, 1
        )  # hsv color with hue 0.66 (blue), saturation as v, value 1
        hex_color = "#%02x%02x%02x" % (
            int(rgb_color[0] * 255),
            int(rgb_color[1] * 255),
            int(rgb_color[2] * 255),
        )
        html += f'<span style="background-color: {hex_color}; border: 1px solid lightgray; font-size: 16px; border-radius: 3px;">{s}</span>'
    if return_string:
        return html
    else:
        display(HTML(html))

def basic_feature_vis(text, feature_index, max_val=0):
    feature_in = encoder.W_enc[:, feature_index]
    feature_bias = encoder.b_enc[feature_index]
    _, cache = model.run_with_cache(text, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
    mlp_acts = cache[utils.get_act_name("post", 0)][0]
    feature_acts = F.relu((mlp_acts - encoder.b_dec) @ feature_in + feature_bias)
    if max_val==0:
        max_val = max(1e-7, feature_acts.max().item())
        # print(max_val)
    # if min_val==0:
    #     min_val = min(-1e-7, feature_acts.min().item())
    return basic_token_vis_make_str(text, feature_acts, max_val)
def basic_token_vis_make_str(strings, values, max_val=None):
    if not isinstance(strings, list):
        strings = model.to_str_tokens(strings)
    values = utils.to_numpy(values)
    if max_val is None:
        max_val = values.max()
    # if min_val is None:
    #     min_val = values.min()
    header_string = f"<h4>Max Range <b>{values.max():.4f}</b> Min Range: <b>{values.min():.4f}</b></h4>"
    header_string += f"<h4>Set Max Range <b>{max_val:.4f}</b></h4>"
    # values[values>0] = values[values>0]/ma|x_val
    # values[values<0] = values[values<0]/abs(min_val)
    body_string = create_html(strings, values, max_value=max_val, return_string=True)
    return header_string + body_string
# display(HTML(basic_token_vis_make_str(tokens[0, :10], mlp_acts[0, :10, 7], 0.1)))
# # %%
# The `with gr.Blocks() as demo:` syntax just creates a variable called demo containing all these components
import gradio as gr
try:
    demos[0].close()
except:
    pass
demos = [None]
def make_feature_vis_gradio(feature_id, starting_text=None, batch=None, pos=None):
    if starting_text is None:
        starting_text = model.to_string(all_tokens[batch, 1:pos+1])
    try:
        demos[0].close()
    except:
        pass
    with gr.Blocks() as demo:
        gr.HTML(value=f"Hacky Interactive Neuroscope for gelu-1l")
        # The input elements
        with gr.Row():
            with gr.Column():
                text = gr.Textbox(label="Text", value=starting_text)
                # Precision=0 makes it an int, otherwise it's a float
                # Value sets the initial default value
                feature_index = gr.Number(
                    label="Feature Index", value=feature_id, precision=0
                )
                # # If empty, these two map to None
                max_val = gr.Number(label="Max Value", value=None)
                # min_val = gr.Number(label="Min Value", value=None)
                inputs = [text, feature_index, max_val]
        with gr.Row():
            with gr.Column():
                # The output element
                out = gr.HTML(label="Neuron Acts", value=basic_feature_vis(starting_text, feature_id))
        for inp in inputs:
            inp.change(basic_feature_vis, inputs, out)
    demo.launch(share=True)
    demos[0] = demo

### Inspecting Top Logits

In [10]:
SPACE = "·"
NEWLINE="↩"
TAB = "→"
def process_token(s):
    if isinstance(s, torch.Tensor):
        s = s.item()
    if isinstance(s, np.int64):
        s = s.item()
    if isinstance(s, int):
        s = model.to_string(s)
    s = s.replace(" ", SPACE)
    s = s.replace("\n", NEWLINE+"\n")
    s = s.replace("\t", TAB)
    return s

def process_tokens(l):
    if isinstance(l, str):
        l = model.to_str_tokens(l)
    elif isinstance(l, torch.Tensor) and len(l.shape)>1:
        l = l.squeeze(0)
    return [process_token(s) for s in l]

def process_tokens_index(l):
    if isinstance(l, str):
        l = model.to_str_tokens(l)
    elif isinstance(l, torch.Tensor) and len(l.shape)>1:
        l = l.squeeze(0)
    return [f"{process_token(s)}/{i}" for i,s in enumerate(l)]

def create_vocab_df(logit_vec, make_probs=False, full_vocab=None):
    if full_vocab is None:
        full_vocab = process_tokens(model.to_str_tokens(torch.arange(model.cfg.d_vocab)))
    vocab_df = pd.DataFrame({"token": full_vocab, "logit": utils.to_numpy(logit_vec)})
    if make_probs:
        vocab_df["log_prob"] = utils.to_numpy(logit_vec.log_softmax(dim=-1))
        vocab_df["prob"] = utils.to_numpy(logit_vec.softmax(dim=-1))
    return vocab_df.sort_values("logit", ascending=False)

### Make Token DataFrame

In [11]:
def list_flatten(nested_list):
    return [x for y in nested_list for x in y]
def make_token_df(tokens, len_prefix=5, len_suffix=1):
    str_tokens = [process_tokens(model.to_str_tokens(t)) for t in tokens]
    unique_token = [[f"{s}/{i}" for i, s in enumerate(str_tok)] for str_tok in str_tokens]

    context = []
    batch = []
    pos = []
    label = []
    for b in range(tokens.shape[0]):
        # context.append([])
        # batch.append([])
        # pos.append([])
        # label.append([])
        for p in range(tokens.shape[1]):
            prefix = "".join(str_tokens[b][max(0, p-len_prefix):p])
            if p==tokens.shape[1]-1:
                suffix = ""
            else:
                suffix = "".join(str_tokens[b][p+1:min(tokens.shape[1]-1, p+1+len_suffix)])
            current = str_tokens[b][p]
            context.append(f"{prefix}|{current}|{suffix}")
            batch.append(b)
            pos.append(p)
            label.append(f"{b}/{p}")
    # print(len(batch), len(pos), len(context), len(label))
    return pd.DataFrame(dict(
        str_tokens=list_flatten(str_tokens),
        unique_token=list_flatten(unique_token),
        context=context,
        batch=batch,
        pos=pos,
        label=label,
    ))

## Loading the Model

In [12]:
model = HookedTransformer.from_pretrained("gelu-1l").to(DTYPES[cfg["enc_dtype"]])
n_layers = model.cfg.n_layers
d_model = model.cfg.d_model
n_heads = model.cfg.n_heads
d_head = model.cfg.d_head
d_mlp = model.cfg.d_mlp
d_vocab = model.cfg.d_vocab

Loaded pretrained model gelu-1l into HookedTransformer
Changing model dtype to torch.float32


## Loading Data

In [13]:
data = load_dataset("NeelNanda/c4-code-20k", split="train")
print(type(data))
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=128)
tokenized_data = tokenized_data.shuffle(42)
all_tokens = tokenized_data["tokens"]
print('all_tokens.shape', all_tokens.shape)

<class 'datasets.arrow_dataset.Dataset'>
all_tokens.shape torch.Size([215402, 128])


# Analysis

## Loading the Autoencoder

There are two runs on separate random seeds, along with a bunch of intermediate checkpoints

In [14]:
auto_encoder_run = "run1" # @param ["run1", "run2"]
encoder = AutoEncoder.load_from_hf(auto_encoder_run)

{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 2048,
 'dict_mult': 8,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'lr': 0.0001,
 'model_batch_size': 512,
 'num_tokens': 2000000000,
 'seed': 52,
 'seq_len': 128}


## Using the Autoencoder

We run the model and replace the MLP activations with those reconstructed from the autoencoder, and get 91% loss recovered

In [15]:
_ = get_recons_loss(num_batches=5, local_encoder=encoder)

loss: 3.2546, recons_loss: 3.7507, zero_abl_loss: 8.8403
Reconstruction Score: 91.12%


## Rare Features Are All The Same

For each feature we can get the frequency at which it's non-zero (per token, averaged across a bunch of batches), and plot a histogram

In [16]:
freqs = get_freqs(num_batches = 50, local_encoder = encoder)

  0%|          | 0/50 [00:00<?, ?it/s]

Num dead tensor(0., device='cuda:0')


In [18]:
# Add 1e-6.5 so that dead features show up as log_freq -6.5
log_freq = (freqs + 10**-6.5).log10()
px.histogram(utils.to_numpy(log_freq), title="Log Frequency of Features", histnorm='percent')

We see that it's clearly bimodal! Let's define rare features as those with freq < 1e-4, and look at the cosine sim of each feature with the average rare feature - we see that almost all rare features correspond to this feature!

In [19]:
is_rare = freqs < 1e-4
rare_enc = encoder.W_enc[:, is_rare]
rare_mean = rare_enc.mean(-1)
px.histogram(utils.to_numpy(rare_mean @ encoder.W_enc / rare_mean.norm() / encoder.W_enc.norm(dim=0)), title="Cosine Sim with Ave Rare Feature", color=utils.to_numpy(is_rare), labels={"color": "is_rare", "count": "percent", "value": "cosine_sim"}, marginal="box", histnorm="percent", barmode='overlay')

## Interpreting A Feature

Let's go and investigate a non rare feature, feature 7

In [20]:
feature_id = 7 # @param {type:"number"}
batch_size = 128 # @param {type:"number"}

print(f"Feature freq: {freqs[7].item():.4f}")

Feature freq: 0.0029


Let's run the model on some text and then use the autoencoder to process the MLP activations

In [21]:
tokens = all_tokens[:batch_size]
_, cache = model.run_with_cache(tokens, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
mlp_acts = cache[utils.get_act_name("post", 0)]
print(type(mlp_acts))
print(cfg['d_mlp'])
mlp_acts_flattened = mlp_acts.reshape(-1, cfg["d_mlp"])
loss, x_reconstruct, hidden_acts, l2_loss, l1_loss = encoder(mlp_acts_flattened)
# This is equivalent to:
# hidden_acts = F.relu((mlp_acts_flattened - encoder.b_dec) @ encoder.W_enc + encoder.b_enc)
print("hidden_acts.shape", hidden_acts.shape)

<class 'torch.Tensor'>
2048
hidden_acts.shape torch.Size([16384, 16384])


We can now sort and display the top tokens, and we see that this feature activates on text like " and I" (ditto for other connectives and pronouns)! It seems interpretable!

**Aside:** Note on how to read the context column:

A line like "·himself·as·democratic·socialist·and|·he|·favors" means that the preceding 5 tokens are " himself as democratic socialist and", the current token is " he" and the next token is " favors".  · are spaces, ↩ is a newline.

This gets a bit confusing for this feature, since the pipe separators look a lot like a capital I


In [22]:
token_df = make_token_df(tokens)
print(hidden_acts[:, feature_id].shape)
token_df["feature"] = utils.to_numpy(hidden_acts[:, feature_id])
token_df.sort_values("feature", ascending=False).head(20).style.background_gradient("coolwarm")

torch.Size([16384])


,str_tokens,unique_token,context,batch,pos,label,feature
6617,·you,·you/89,·Don·Jaime·I·and|·you|·will,51,89,51/89,1.560131
11525,·it,·it/5,<|BOS|>in·is·True·and|·it|·is,90,5,90/5,1.155806
13829,·it,·it/5,"<|BOS|>·image·file,·and|·it|·should",108,5,108/5,0.965699
6551,·it,·it/23,·meets·the·new·one·and|·it|’,51,23,51/23,0.900990
4314,·she,·she/90,"·life,·growing·up,|·she|·was",33,90,33/90,0.463120
5588,·he,·he/84,·through·grace·—·and·now|·he|·shares,43,84,43/84,0.385578
9689,·what,·what/89,·to·perceive·buyer·behavior·and|·what|·it,75,89,75/89,0.290015
6552,’,’/24,·the·new·one·and·it|’|s,51,24,51/24,0.253980
4326,·hog,·hog/102,’s·row·crop·and|·hog|·farm,33,102,33/102,0.233893
6618,·will,·will/90,·Jaime·I·and·you|·will|·arrive,51,90,51/90,0.229745


It's easy to misread evidence like the above, so it's useful to take some text and edit it and see how this changes the model's activations. Here's a hacky interactive tool to play around with some text.

In [23]:
model.cfg

HookedTransformerConfig:
{'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 64,
 'd_mlp': 2048,
 'd_model': 512,
 'd_vocab': 48262,
 'd_vocab_out': 48262,
 'default_prepend_bos': True,
 'device': device(type='cuda'),
 'dtype': torch.float32,
 'eps': 1e-05,
 'experts_per_token': None,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.035355339059327376,
 'load_in_4bit': False,
 'model_name': 'GELU_1L512W_C4_Code',
 'n_ctx': 1024,
 'n_devices': 1,
 'n_heads': 8,
 'n_key_value_heads': None,
 'n_layers': 1,
 'n_params': 3145728,
 'normalization_type': 'LNPre',
 'num_experts': None,
 'original_architecture': 'neel',
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'standard',
 'post_embedding_ln': False,
 'rotary_adjacent_pairs': False,
 'rotary_base': 10000,
 

In [24]:
s = "The 1899 Kentucky gubernatorial election was held on November 7, 1899. The Republican incumbent, William Bradley, was term-limited. The Democrats chose William Goebel. Republicans nominated William Taylor. Taylor won by a vote of 193,714 to 191,331. The vote was challenged on grounds of voter fraud, but the Board of Elections, though stocked with pro-Goebel members, certified the result. Democratic legislators began investigations, but before their committee could report, Goebel was shot by an unknown assassin (event pictured) on January 30, 1900. Democrats voided enough votes to swing the election to Goebel, Taylor was deposed, and Goebel was sworn into office on January 31. He died on February 3. The lieutenant governor of Kentucky, J. C. W. Beckham, became governor, and battled Taylor in court. Beckham won on appeal, and Taylor fled to Indiana, fearing arrest as an accomplice. The only persons convicted in connection with the killing were later pardoned; the assassin's identity remains a mystery"
t = model.to_tokens(s)
print(t)

tensor([[    1,   510,   338,    26,    27,    27, 17846,   306, 22621, 25060,
          5962,   369,  2828,   328,  4452,   813,    14,   338,    26,    27,
            27,    16,   380,  8558, 34001,    14,  7060, 27032,    14,   369,
          1283,    15, 15430,    16,   380, 10540,  9448,  7060,  3497,  2204,
           294,    16, 11564, 24709,  7060, 10978,    16, 10978,  1856,   407,
           248,  6101,   274,   338,    27,    21,    14,    25,    19,    22,
           282,   338,    27,    19,    14,    21,    21,    19,    16,   380,
          6101,   369, 14459,   328,  9643,   274, 23044,  8879,    14,   532,
           254,  5816,   274,   444,  9628,    14,  2101,  5574,   265,   343,
           355,    15,  6644,  2204,   294,  2672,    14, 17551,   254,   898,
            16,  9659, 35835,  3297, 13625,    14,   532,  1063,   615,  9106,
           807,  1280,    14,  3497,  2204,   294,   369,  4950,   407,   272,
          7011, 44280,   314,  7833, 34609,    11,  

In [25]:

starting_text = "Hero and I will head to Samantha and Mark's, then he and she will. Then I or you" # @param {type:"string"}
make_feature_vis_gradio(feature_id, starting_text)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://101141a02ba551ea2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ConnectTimeout: _ssl.c:990: The handshake operation timed out

A final piece of evidence: This is a one layer model, so the neurons can only matter by directly impacting the final logits! We can directly look at how the decoder weights for this feature affect the logits, and see that it boosts `'ll`! This checks out, I and he'll etc is a common construction.

In [26]:
logit_effect = encoder.W_dec[feature_id] @ model.W_out[0] @ model.W_U
create_vocab_df(logit_effect).head(20).style.background_gradient("coolwarm")

,token,logit
1782,'ll,1.875001
6835,eding,1.558904
5440,·certainly,1.522172
3409,·hope,1.521111
29310,OULD,1.508625
4931,·wouldn,1.483282
44469,cheon,1.435624
7754,·definitely,1.397461
1606,·seem,1.394357
41304,·sincerely,1.363966


In [27]:
# Let's try to pass through inputs with the same label from a sentiment analysis dataset to see what features are activated the most
# We'll use a Twitter sentiment analysis dataset

import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

# Example for turning pandas DF to huggingface Dataset
df = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]})
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

# Load the dataset as an HG dataset
dataset = load_dataset('csv', data_files="twitter_training.csv", split="train")
dataset = dataset.rename_column('Tweet content', 'text')
dataset = dataset.filter(lambda x: x['text'] != None)
print(dataset)

# Take out just the Tweet content
#tweet_content = dataset['Tweet content']
# Rename the column to 'text' to match the model
#tweet_content = tweet_content.rename_column('Tweet content', 'text')

tokenized_data = utils.tokenize_and_concatenate(dataset, model.tokenizer, max_length=128)
tokenized_data = tokenized_data.shuffle(seed=42)
all_tokens = tokenized_data["tokens"]
print('all_tokens.shape', all_tokens.shape)

batch_size = 128
example_tokens = all_tokens[:batch_size] 

_, cache = model.run_with_cache(example_tokens, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
mlp_acts = cache[utils.get_act_name("post", 0)]
mlp_acts_flattened = mlp_acts.reshape(-1, cfg["d_mlp"])

loss, x_reconstruct, hidden_acts, l2_loss, l1_loss = encoder(mlp_acts_flattened)
print("hidden_acts.shape", hidden_acts.shape)

Dataset({
    features: ['Tweet ID', 'entity', 'sentiment', 'text'],
    num_rows: 73996
})
all_tokens.shape torch.Size([17597, 128])
hidden_acts.shape torch.Size([16384, 16384])


In [28]:
token_df = make_token_df(example_tokens)
token_df["feature"] = utils.to_numpy(hidden_acts[:, feature_id])
token_df.sort_values("feature", ascending=False).head(20).style.background_gradient("coolwarm")

,str_tokens,unique_token,context,batch,pos,label,feature
7772,·I,·I/92,Rainbow6Game·and|·I|·just,60,92,60/92,1.827389
5743,·I,·I/111,BMM·is·dumb·and|·I|·hate,44,111,44/111,1.720643
16021,·we,·we/21,"performance·graphics·processors,·and|·we|·still",125,21,125/21,1.713331
4067,·we,·we/99,"indragosa,·and|·we|·didn",31,99,31/99,1.695874
4016,·we,·we/48,ova·in·the·trailer·and|·we|·didn,31,48,31/48,1.686774
3317,·I,·I/117,AC<|EOS|>My·husband·and|·I|·laid,25,117,25/117,1.678548
5726,·I,·I/94,t·want·to·play·and|·I|·leave,44,94,44/94,1.672901
4617,·they,·they/9,·This·means·that·he·and|·they|·manage,36,9,36/9,1.654053
3268,·I,·I/68,·shit<|EOS|>My·husband·and|·I|·lay,25,68,25/68,1.613762
15137,·they,·they/33,·players·in·PUBG·and|·they|·started,118,33,118/33,1.599694


In [29]:
# For each set of tokens, check which features are activated the most

def argsort(seq, reverse=False):
    return sorted(range(len(seq)), key=seq.__getitem__, reverse=reverse)

batch_df = token_df
# Go through every feature and keep track of the most activated feature
activations = [None for _ in range(encoder.d_hidden)]
mean_activations = [None for _ in range(encoder.d_hidden)]
for feature_id in range(encoder.d_hidden):

    feature_act = utils.to_numpy(hidden_acts[:, feature_id])
    
    activations[feature_id] = feature_act
    # Measure max activation based on average activation (for now)
    mean_activations[feature_id] = feature_act.mean()

# Sort the features by the mean of the max activations
sorted_feature_idxs = argsort(mean_activations, reverse=True)

In [30]:
# Visualize the MOST activated feature (based on whatever metric we used)
highest_act_idx = sorted_feature_idxs[0]
print(highest_act_idx)
batch_df['feature'] = utils.to_numpy(hidden_acts[:, highest_act_idx])
batch_df.sort_values("feature", ascending=False).head(20).style.background_gradient("coolwarm")

12344


,str_tokens,unique_token,context,batch,pos,label,feature
12127,·I,·I/95,"·It's·not·bad,|·I|'m",94,95,94/95,3.002021
12128,'m,'m/96,"'s·not·bad,·I|'m|·sure",94,96,94/96,2.902968
12124,·not,·not/92,··.·.·It's|·not|·bad,94,92,94/92,2.769646
3059,·that,·that/115,·is·not·the·first·time|·that|·the,23,115,23/115,2.656271
12122,·It,·It/90,·gameplay·of··.·.|·It|'s,94,90,94/90,2.649644
11241,?,?/105,·heart·ain’t·enough|?|·Are,87,105,87/105,2.628554
12130,·I,·I/98,"·bad,·I'm·sure|·I|'ll",94,98,94/98,2.566261
6257,·being,·being/113,·dude<|EOS|>btw·annoying|·being|·bab,48,113,48/113,2.565367
11240,·enough,·enough/104,·my·heart·ain’t|·enough|?,87,104,87/104,2.532764
12123,'s,'s/91,·of··.·.·It|'s|·not,94,91,94/91,2.503203


In [31]:
logit_effect = encoder.W_dec[highest_act_idx] @ model.W_out[0] @ model.W_U
create_vocab_df(logit_effect).head(20).style.background_gradient("coolwarm")


,token,logit
37060,·blogging,1.112262
48144,·Parenthood,1.066725
19702,·Blog,1.012843
32012,fiction,1.009526
47025,·Wrote,0.976965
22805,·crap,0.955696
46981,Blog,0.952527
40340,·bloggers,0.938339
14697,·pixels,0.931579
40098,·Debian,0.925393


In the cell above we can see that for our randomly sampled batch of data this is the feature that has the highest mean activation. Unfortunately, it doesn't seem to mean much.

In [32]:
# Now that we know how to do this let's separate our data into posiitive, neutral, and negative sentiments
# For now just try negative
negative_segment = dataset.filter(lambda x: x['sentiment'] == 'Negative')

tokenized_negative = utils.tokenize_and_concatenate(negative_segment, model.tokenizer, max_length=128)
tokenized_negative = tokenized_negative.shuffle(seed=42)
all_tokens_negative = tokenized_negative["tokens"]

batch_size = 256
batch_negative = all_tokens_negative[:batch_size]

_, cache = model.run_with_cache(batch_negative, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
mlp_acts = cache[utils.get_act_name("post", 0)]
mlp_acts_flattened = mlp_acts.reshape(-1, cfg["d_mlp"])

loss, x_reconstruct, hidden_acts, l2_loss, l1_loss = encoder(mlp_acts_flattened)

token_df_negative = make_token_df(batch_negative)
batch_df = token_df_negative
# Go through every feature and keep track of the most activated feature
activations = [None for _ in range(encoder.d_hidden)]
mean_activations = [None for _ in range(encoder.d_hidden)]
for feature_id in range(encoder.d_hidden):

    feature_act = utils.to_numpy(hidden_acts[:, feature_id])
    
    activations[feature_id] = feature_act
    # Measure max activation based on average activation (for now)
    mean_activations[feature_id] = feature_act.mean()

# Sort the features by the mean of the max activations
sorted_feature_idxs = argsort(mean_activations, reverse=True)

In [33]:
# Visualize the MOST activated feature (based on whatever metric we used)
highest_act_idx = sorted_feature_idxs[0]
print(highest_act_idx)
batch_df['feature'] = utils.to_numpy(hidden_acts[:, highest_act_idx])
batch_df.sort_values("feature", ascending=False).head(20).style.background_gradient("coolwarm")

12344


,str_tokens,unique_token,context,batch,pos,label,feature
30060,·in,·in/108,·looks·like·someone·stuck·it|·in|·the,234,108,234/108,2.808305
26330,...,.../90,·didn’t·age·well|...|@,205,90,205/90,2.707428
32560,·I,·I/48,·of·fifa·tonight·and|·I|’,254,48,254/48,2.691983
30059,·it,·it/107,5·looks·like·someone·stuck|·it|·in,234,107,234/107,2.661819
30061,·the,·the/109,·like·someone·stuck·it·in|·the|·microwave,234,109,234/109,2.596568
3672,?,?/88,·Demon·Hunter·so·dirty|?|·.,28,88,28/88,2.593854
30076,·in,·in/124,·like·someone·has·stuck·it|·in|·the,234,124,234/124,2.525397
21471,·is,·is/95,",·the·goddamn·match·making|·is|·broken",167,95,167/95,2.483955
22004,·with,·with/116,lands·something·seems·very·broken|·with|·M,171,116,171/116,2.476697
32564,.,./52,·and·I’m·done|.|·Press,254,52,254/52,2.447253


In [34]:
logit_effect = encoder.W_dec[highest_act_idx] @ model.W_out[0] @ model.W_U
create_vocab_df(logit_effect).head(20).style.background_gradient("coolwarm")

,token,logit
37060,·blogging,1.112262
48144,·Parenthood,1.066725
19702,·Blog,1.012843
32012,fiction,1.009526
47025,·Wrote,0.976965
22805,·crap,0.955696
46981,Blog,0.952527
40340,·bloggers,0.938339
14697,·pixels,0.931579
40098,·Debian,0.925393


Ok so this doesn't seem to mean much either... but that is to be expected since the training data for the SAE is probably not sufficient, and the context of all the negative-labelled data is probably not very cohesive.


In [74]:
# Refactor the feature search process into a function

def argsort(seq, reverse=False):
        return sorted(range(len(seq)), key=seq.__getitem__, reverse=reverse)

def feature_search(dataset, model, autoencoder, batch_size=128, token_length=128):
    '''
    Search through features of a pre-trained SAE.
    'dataset' must first be loaded as an HuggingFace Dataset:
    from 
    dataset = load_dataset('file_type', data_files="local_file.file_type", split="whatever_split")
    '''
    tokenized_data = utils.tokenize_and_concatenate(dataset, model.tokenizer, max_length=token_length)
    tokenized_data = tokenized_data.shuffle(seed=42)
    all_tokens = tokenized_data["tokens"]

    example_tokens = all_tokens[:batch_size] 

    _, cache = model.run_with_cache(example_tokens, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
    mlp_acts = cache[utils.get_act_name("post", 0)]
    mlp_acts_flattened = mlp_acts.reshape(-1, cfg["d_mlp"])

    loss, x_reconstruct, hidden_acts, l2_loss, l1_loss = autoencoder(mlp_acts_flattened)
    print("hidden_acts.shape", hidden_acts.shape)

    batch_df = make_token_df(example_tokens)

    # Go through every feature and keep track of the most activated feature
    activations = [None for _ in range(autoencoder.d_hidden)]
    mean_activations = [None for _ in range(autoencoder.d_hidden)]
    for feature_id in range(autoencoder.d_hidden):

        feature_act = utils.to_numpy(hidden_acts[:, feature_id])

        activations[feature_id] = feature_act

        # Measure most activation based on average of top 3 activations
        # NOTE: fix here, this is not taking the largest activations
        sorted_feature_idxs = np.argsort(feature_act)
        sorted_feature_act = feature_act[sorted_feature_idxs]
        mean_activations[feature_id] = sorted_feature_act[-3:].mean()

        # Measure most activation based on average activation (for now)
        # mean_activations[feature_id] = feature_act.mean()

    # Sort the features by the mean of the max activations
    sorted_feature_idxs = argsort(mean_activations, reverse=True)
    #print(mean_activations[sorted_feature_idxs[0]])

    # Visualize the MOST activated feature (based on whatever metric we used)
    highest_act_idx = sorted_feature_idxs[0]
    batch_df['feature'] = utils.to_numpy(hidden_acts[:, highest_act_idx])
    

    logit_effect = autoencoder.W_dec[highest_act_idx] @ model.W_out[0] @ model.W_U
    

    return highest_act_idx, batch_df, logit_effect


# Just to make sure the function works
feature_id, batch_df, logit_effect = feature_search(negative_segment, model, encoder, batch_size=128)
print(feature_id)

hidden_acts.shape torch.Size([16384, 16384])
4022


In [75]:
#print('Feature that activates the most on this batch')
batch_df.sort_values("feature", ascending=False).head(20).style.background_gradient("coolwarm")

,str_tokens,unique_token,context,batch,pos,label,feature
12026,<|EOS|>,<|EOS|>/122,·me·redeem·the·skin.|<|EOS|>|@,93,122,93/122,8.957706
3925,<|EOS|>,<|EOS|>/85,Overwatch·No·Thanks.|<|EOS|>|Hey,30,85,30/85,8.843098
14758,<|EOS|>,<|EOS|>/38,·console·(console·player).|<|EOS|>|@,115,38,115/38,8.623754
3870,<|EOS|>,<|EOS|>/30,Overwatch·:)·Thanks.|<|EOS|>|Hey,30,30,30/30,8.460874
11203,<|EOS|>,<|EOS|>/67,·people·that·is·lost.|<|EOS|>|Lo,87,67,87/67,8.319877
15680,<|EOS|>,<|EOS|>/64,·access·to·health·care.|<|EOS|>|@,122,64,122/64,8.299183
15148,<|EOS|>,<|EOS|>/44,·the·game·like·this!!!|<|EOS|>|@,118,44,118/44,8.247939
1585,<|EOS|>,<|EOS|>/49,",·this·place·is·terrible|<|EOS|>|@",12,49,12/49,8.231568
8556,<|EOS|>,<|EOS|>/108,·a·good·list·anywhere.|<|EOS|>|Indian,66,108,66/108,8.199583
13136,<|EOS|>,<|EOS|>/80,·out·of·the·selection.|<|EOS|>|@,102,80,102/80,8.198583


In [70]:
#print('\n\n')
#print('Top 20 tokens that activate the feature the most')
create_vocab_df(logit_effect).head(20).style.background_gradient("coolwarm")

,token,logit
13761,Discussion,3.805068
20464,Welcome,3.420099
31883,�,3.340961
29838,Born,3.209161
22037,Women,3.191888
23426,Police,3.188007
11765,Hello,3.153953
8043,Hey,3.077714
31760,Published,3.077039
44913,Meet,3.047427


In [76]:
positive_segment = dataset.filter(lambda x: x['sentiment'] == 'Positive')

feature_id_positive, batch_df_positive, logit_effect_positive = feature_search(positive_segment, model, encoder, token_length=256)
print(feature_id_positive)

hidden_acts.shape torch.Size([32768, 16384])
16350


In [72]:
batch_df_positive.sort_values("feature", ascending=False).head(20).style.background_gradient("coolwarm")

,str_tokens,unique_token,context,batch,pos,label,feature
21955,<|EOS|>,<|EOS|>/195,·durable·than·this·garbage.|<|EOS|>|The,85,195,85/195,8.954926
21983,<|EOS|>,<|EOS|>/223,·than·this·thrash.|<|EOS|>|War,85,223,85/223,8.812435
13693,<|EOS|>,<|EOS|>/125,"·streaming·test,·though.|<|EOS|>|Really",53,125,53/125,8.771532
4768,<|EOS|>,<|EOS|>/160,·in·the·next·one).|<|EOS|>|HA,18,160,18/160,8.751431
22009,<|EOS|>,<|EOS|>/249,·consistent·than·another·one.|<|EOS|>|War,85,249,85/249,8.676452
4860,<|EOS|>,<|EOS|>/252,·at·the·next·one).|<|EOS|>|HA,18,252,18/252,8.619102
30602,<|EOS|>,<|EOS|>/138,·and·hitting·every·shot.|<|EOS|>|Mm,119,138,119/138,8.579471
27897,<|EOS|>,<|EOS|>/249,·Port·PVP·card.|<|EOS|>|here,108,249,108/249,8.500069
14523,<|EOS|>,<|EOS|>/187,·taken·such·a·step.|<|EOS|>|At,56,187,56/187,8.463744
4812,<|EOS|>,<|EOS|>/204,·kills·them·next·time).|<|EOS|>|HA,18,204,18/204,8.458066


In [73]:
create_vocab_df(logit_effect_positive).head(20).style.background_gradient("coolwarm")

,token,logit
13761,Discussion,2.970305
23426,Police,2.656853
20464,Welcome,2.625100
31883,�,2.598711
31760,Published,2.527143
44913,Meet,2.515903
18327,London,2.492368
22037,Women,2.470967
22094,China,2.438622
29838,Born,2.432478


In [33]:
# Let's try a sentiment analysis dataset from Huuggingface
# Using the news_sentiment_newsmtsc dataset

import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

# Load the dataset
dataset = load_dataset("news_sentiment_newsmtsc", split="train")

_, cache = model.run_with_cache(example_tweet_tokens, stop_at_layer=1, names_filter=utils.get_act_name("post", 0))
mlp_acts = cache[utils.get_act_name("post", 0)][0]
mlp_acts_flattened = mlp_acts.reshape(-1, cfg["d_mlp"])

loss, x_reconstruct, hidden_acts, l2_loss, l1_loss = encoder(mlp_acts_flattened)
print("hidden_acts.shape", hidden_acts.shape)


num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


                                                text
0  im getting on borderlands and i will murder yo...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

ValueError: Columns ['tokens'] not in the dataset. Current columns in the dataset: ['text']